### Базовый алгоритм RAG
(обработка нормативной документации по строительству объектов)

### Загрузка документов

In [1]:
import os
import re
from langchain.schema import Document

root_folder = "docs"

# Функция для извлечения номера страницы из имени файла
def extract_page_number(file_name):
    match = re.search(r'page_(\d+)', file_name)
    if match:
        return int(match.group(1))
    return None

# Переменная для хранения всех документов
documents = []

# Проходим по каждой папке внутри root_folder
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    
    if os.path.isdir(folder_path):        
        # Ищем подпапку text_from_pdf в текущей папке
        text_from_pdf_folder = os.path.join(folder_path, "text_from_pdf")
        if os.path.isdir(text_from_pdf_folder):  # Проверяем, существует ли подпапка text_from_pdf
            # Собираем все txt файлы и их номера страниц
            files_with_pages = []
            for file_name in os.listdir(text_from_pdf_folder):
                file_path = os.path.join(text_from_pdf_folder, file_name)
                
                # Проверяем, что это txt-файл
                if file_name.endswith(".txt"): 
                    page_number = extract_page_number(file_name)
                    if page_number is not None:
                        files_with_pages.append((file_name, page_number))
            
            # Сортируем файлы по номеру страницы
            sorted_files = sorted(files_with_pages, key=lambda x: x[1])
            
            # Проходим по отсортированным файлам
            for file_name, page_number in sorted_files:
                file_path = os.path.join(text_from_pdf_folder, file_name)
                
                with open(file_path, "r", encoding="utf-8") as file:
                    page_content = file.read()

                # Создаем объект Document с метаданными
                doc = Document(
                    page_content=page_content,
                    metadata={
                        'source': folder_name,  # Название основной папки
                        'page': page_number     # Номер страницы
                    }
                )
                
                # Добавляем документ в список
                documents.append(doc)

### Разбивка документов на чанки 

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

### Подгружаем LLM

In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

llm = Ollama(model="llama3.1")
embedding_model = OllamaEmbeddings(model="llama3.1")

### Строим prompt-инструкцию для LLM

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы по нормативной документации по строительству объектов. 
                                          Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос.
                                          Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")

### Индексируем чанки 

In [11]:
splits[:20]

[Document(page_content='МИНИСТЕРСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПО ДЕЛАМ ГРАЖДАНСКОЙ  \nОБОРОНЫ, ЧРЕЗВЫЧАЙНЫМ СИТУАЦИЯМ И ЛИКВИДАЦИИ  \nПОСЛЕДСТВИЙ СТИХИЙНЫХ БЕДСТВИЙ  \n \nПРИКАЗ  \nот 19 марта 2020 г. N 194  \n \nОБ УТВЕРЖДЕНИИ СВОДА ПРАВИЛ  \nСП 1.13130 "СИСТЕМЫ ПРОТИВОПОЖАРНОЙ ЗАЩИТЫ.  \nЭВАКУАЦИОННЫЕ ПУТИ И ВЫХОДЫ"  \n \nВ соответствии с Федеральным законом  от 22 июля 2008 г. N 123 -ФЗ "Технический регламент \nо требованиях пожарной безопасности" <1>, Указом  Президента Российской Федерации от 11 июля \n2004 г. N 868 "Вопросы Министерства Российской Федерации по делам гражданской обороны, \nчрезвычайным ситуациям и ликвид ации последствий стихийных бедствий" <2> и постановлением  \nПравительства Российской Федерации от 1 июля 2016 г. N 624 "Об утверждении Правил разработки, \nутверждения, о публикования, изменения и отмены сводов правил" <3> приказываю:  \n--------------------------------  \n<1> Собрание законодательства Российской Федерации, 2008, N 30, ст. 3579; 2018, N 53, ст. \n846

In [12]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(documents=splits[:20], embedding=embedding_model)
# сохранение
db.save_local("path_to_faiss_index")

In [13]:
from langchain.vectorstores import FAISS

# Загружаем FAISS индекс из файла с разрешением 
db = FAISS.load_local(
    "path_to_faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Устанавливаем ретривер для поиска контекста
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

In [ ]:
# # временное хранение
# from langchain.vectorstores import FAISS
# db = FAISS.from_documents(documents=splits, embedding=embedding_model)

# # устанавливаем ретриевер для поиска контекста
# retriever = db.as_retriever(
#     search_type="similarity",
#     search_kwargs={'k': 3}
# )

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke("Что такое МГН?")

In [17]:
answer

'МГН - маломобильные группы населения.'